In [1]:
import pandas as pd
import numpy as np

In [2]:
inventory = pd.read_csv("MS-b1-inventory.csv")
sell_logs = pd.read_csv("MS-b1-sell.csv")

In [3]:
supply = sell_logs.copy()
supply['apple'] = supply.groupby('date')['sku_num'].transform(lambda x: x[x.str.contains('MS-b1-ap-')].count())
supply['pen'] = supply.groupby('date')['sku_num'].transform(lambda x: x[x.str.contains('MS-b1-pe-')].count())
supply = supply.drop(columns='sku_num')
supply = supply.drop_duplicates(["date"]).reset_index().drop(columns='index')


In [4]:
len(supply)

1853

In [5]:
supply.head()

,date,apple,pen
0,2006-01-01,1815,156
1,2006-01-02,1862,143
2,2006-01-03,1880,171
3,2006-01-04,1797,153
4,2006-01-05,1942,133


In [6]:
inventory.head()

,date,apple,pen
0,2006-01-31,12157,811
1,2006-02-28,29859,2280
2,2006-03-31,42135,3317
3,2006-04-30,55235,4094
4,2006-05-31,66963,5158


In [7]:
sell_logs.head()

,date,sku_num
0,2006-01-01,MS-b1-ap-48914c5b-14d2-4b20-bdaf-b2ff5d9f4f0c
1,2006-01-01,MS-b1-ap-6baf7287-3e6a-4728-a3b1-8613de51eef8
2,2006-01-01,MS-b1-ap-83d7b005-c7d9-4deb-93a2-a8f7606d02b5
3,2006-01-01,MS-b1-ap-9099c5de-028d-4b07-97be-6f6532b8674a
4,2006-01-01,MS-b1-ap-ee0ce843-a979-4798-a542-75a41d1610c3


##2. Сделаем из логов красивую табличку
- Из логов достанем информацию, продали яблоко или карандаш
- Сгруппируем по дате, для каждой даты считая общее количество проданных товаров

In [8]:
months = [29, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
copy_sup = supply.copy()
print(type(supply['date'][0]))
prev_m = '01'
prev_val_apples, prev_val_pens = 0, 0
for curr_d in supply['date']:
  sold_apples = supply[supply['date'] == curr_d]['apple'].values[0]
  sold_pens = supply[supply['date'] == curr_d]['pen'].values[0]

  if curr_d.split('-')[1] != prev_m:
    dd = curr_d[:4]+'-'+str(prev_m)+'-'+ (str(months[int(prev_m)]) if not(prev_m=='02' and int(curr_d[:4])%4==0) else str(months[0]))

    inventory_apples = inventory[inventory['date'] == dd]['apple'].values[0]
    prev_val_apples = inventory_apples - sold_apples
    supply.loc[supply['date'] == curr_d, 'apple'] = supply[supply['date'] == curr_d]['apple'].replace(sold_apples, prev_val_apples)

    inventory_pens = inventory[inventory['date'] == dd]['pen'].values[0]
    prev_val_pens = inventory_pens - sold_pens
    supply.loc[supply['date'] == curr_d, 'pen'] = supply[supply['date'] == curr_d]['pen'].replace(sold_pens, prev_val_pens)

    prev_m = curr_d[5:7]
  else:
    prev_val_apples -= sold_apples
    supply.loc[supply['date'] == curr_d, 'apple'] = supply[supply['date'] == curr_d]['apple'].replace(sold_apples, prev_val_apples)

    prev_val_pens -= sold_pens
    supply.loc[supply['date'] == curr_d, 'pen'] = supply[supply['date'] == curr_d]['pen'].replace(sold_pens, prev_val_pens)




<class 'str'>


In [11]:
supply.head(1000)

,date,apple,pen
0,2006-01-01,-1815,-156
1,2006-01-02,-3677,-299
2,2006-01-03,-5557,-470
3,2006-01-04,-7354,-623
4,2006-01-05,-9296,-756
...,...,...,...
995,2008-09-22,379842,29790
996,2008-09-23,377962,29636
997,2008-09-24,376054,29509
998,2008-09-25,374190,29367


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [10]:
sell_logs["is_apple"] = sell_logs["sku_num"].str.startswith("MS-b1-ap")
sell_logs["is_pen"] = sell_logs["sku_num"].str.startswith("MS-b1-pe")

In [12]:
sell = pd.DataFrame()
sell["apple"] = sell_logs.groupby("date").is_apple.sum()
sell["pen"] = sell_logs.groupby("date").is_pen.sum()

In [13]:
sell.head()

,apple,pen
date,,
2006-01-01,1815,156
2006-01-02,1862,143
2006-01-03,1880,171
2006-01-04,1797,153
2006-01-05,1942,133
